In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from decimal import Decimal, Context, ROUND_HALF_EVEN, ROUND_DOWN, ROUND_FLOOR, ROUND_HALF_UP, setcontext, localcontext
from itertools import permutations

class MyDecimal:
    def __init__(self, decimal_instance):
        if not isinstance(decimal_instance, Decimal):
            raise ValueError("MyDecimal инициализируется напрямую инстансом Decimal")
        self._decimal = decimal_instance
        self._sign, self._digits, self._exponent = self._decimal.as_tuple()
        self._hex_mantissa = self.get_hex_mantissa_uints32_from_digits()
        self._hex_mantissa_6 = self.get_hex_mantissa_six_uints32_from_digits()
        self._binary_service_part = self.get_binary_service_part()
        self._hex_service_part =  self.get_hex_service_part()
        self.full_hex_repr = self.get_full_hex_repr()
        self.decimal_repr = self.get_decimal_repr()
        
    @classmethod
    def from_string(cls, input_string):
        return cls(Decimal(input_string, Context(prec=29, rounding=ROUND_HALF_EVEN)))

    @classmethod
    def from_decimal(cls, decimal_instance):
        return cls(decimal_instance)

    
    def get_hex_mantissa_uints32_from_digits(self):
        integer_value = int("".join([str(i) for i in self._digits]))

        low = integer_value & 0xFFFFFFFF
        mid = (integer_value >> 32) & 0xFFFFFFFF
        high = (integer_value >> 64) & 0xFFFFFFFF
        
        return f"0x{low:08X}, 0x{mid:08X}, 0x{high:08X}"

    def get_hex_mantissa_six_uints32_from_digits(self):
        integer_value = int("".join([str(i) for i in self._digits]))
    
        p1 = integer_value & 0xFFFFFFFF
        p2 = (integer_value >> 32) & 0xFFFFFFFF
        p3 = (integer_value >> 64) & 0xFFFFFFFF
        p4 = (integer_value >> 96) & 0xFFFFFFFF
        p5 = (integer_value >> 128) & 0xFFFFFFFF
        p6 = (integer_value >> 160) & 0xFFFFFFFF
        
        hex_parts = [
            f"0x{p1:08X}",
            f"0x{p2:08X}",
            f"0x{p3:08X}",
            f"0x{p4:08X}",
            f"0x{p5:08X}",
            f"0x{p6:08X}"
        ]
        
        return ", ".join(hex_parts)

    def get_binary_service_part(self):
        sign_part = str(self._sign)
        scale_part = bin(abs(self._exponent))[2:].zfill(8)
        
        service_part_string = f"0b{sign_part}{7*'0'}{scale_part}{16*'0'}"
        return service_part_string
    
    def get_hex_service_part(self):
        integer_value = int(self._binary_service_part[2:], 2)
        hex_str = f"0x{integer_value:08X}"
        return hex_str

    def get_full_hex_repr(self):
        return f"{self._hex_mantissa}, {self._binary_service_part}"


    def get_decimal_repr(self):
        digit_string = "".join([str(i) for i in self._digits])
        sign_string = '-' if self._sign else ''
        exp_abs = abs(self._exponent)
        if exp_abs == 0:
            return sign_string + digit_string
        elif len(digit_string) <= exp_abs:
            return sign_string + '0.' + '0' * (exp_abs - len(digit_string)) + digit_string
        else:
            return sign_string+digit_string[:-exp_abs] + '.' + digit_string[-exp_abs:]

    def __str__(self):
        return str(self._decimal)

    def __repr__(self):
        return f"MyDecimal('{self._decimal}')"


def normalize_exponent(input_decimal):
   
    sign, digits, exponent = input_decimal._decimal.as_tuple()
    if exponent > 0:
        new_raw_decimal = normalize_positive_exponent(input_decimal._decimal)
    elif exponent < -28:
        new_raw_decimal = normalize_to_min_exponent(input_decimal._decimal)
    else:
         new_raw_decimal = input_decimal._decimal
    return MyDecimal(new_raw_decimal)
    

def normalize_positive_exponent(input_raw_decimal):
    sign, digits, exponent = input_raw_decimal.as_tuple()
    digits = digits + (0,) * exponent
    exponent = 0
    normalized = Decimal((sign, digits, exponent))
    return normalized
    

def normalize_to_min_exponent(input_raw_decimal):
    min_exponent=-28
    sign, digits, exponent = input_raw_decimal.as_tuple()
    digits_to_remove = min_exponent - exponent
    
    new_digits = digits[:max(0, len(digits) - digits_to_remove)]
    
    new_digits = new_digits + (0,) * max(0, digits_to_remove - len(digits))
        
    normalized = Decimal((sign, new_digits, min_exponent))
    
    return normalized

## Mantissa operations

### powers of 10

In [ ]:
def get_text(scale):
    num = 10**scale
    curly_brace = "{"
    curly_brace_closing =  "}"
    dec = MyDecimal.from_string(f"{str(num)}")
    text = f"{curly_brace}{dec._hex_mantissa_6}{curly_brace_closing}, // 10^{scale}" #// 10^{scale}
    return text

big_text = "\n".join([get_text(i) for i in range(58)])

with open('hm.txt', 'w') as file:
    chars_written_1 = file.write(big_text)

### addition

In [ ]:
tested_function_name = "long_mantissas_addition"
first_input = MyDecimal.from_string("945734349573434345343495873493453475983459384593457")
second_input = MyDecimal.from_string('945734349573434345343495873493453475983459384593457')
setcontext(Context(prec=60, rounding=ROUND_HALF_EVEN))

expected_result = MyDecimal(first_input._decimal + second_input._decimal)

curly_brace = "{"
curly_brace_closing =  "}"
curly_braces = "{{"
curly_braces_closing =  "}}"


total_text = f"""
START_TEST(test_{tested_function_name}) {curly_brace}
// {first_input.decimal_repr}
uint32_t term_1[6] = {curly_brace}{first_input._hex_mantissa_6}{curly_brace_closing};
// {second_input.decimal_repr}
uint32_t term_2[6] = {curly_brace}{second_input._hex_mantissa_6}{curly_brace_closing};
// {expected_result.decimal_repr}
uint32_t expected_sum[6] = {curly_brace}{expected_result._hex_mantissa_6}{curly_brace_closing};

uint32_t sum[6] = {curly_brace}0b0, 0b0, 0b0, 0b0, 0b0, 0b0{curly_brace_closing};

int overflow = add_long_mantissas(term_1, term_2, sum, 6);

ck_assert_mem_eq(sum, expected_sum, sizeof(uint32_t) * 6);
ck_assert_int_eq(overflow, 0);
{curly_brace_closing}
END_TEST
    """
print(total_text)




### substraction

In [ ]:
tested_function_name = "long_mantissas_substraction"
first_input = MyDecimal.from_string("1231243453434598789456645745683034558034321234564564354")
second_input = MyDecimal.from_string("85437594753753753975345645675675767000097534985495689")
setcontext(Context(prec=60, rounding=ROUND_HALF_EVEN))

expected_result = MyDecimal(first_input._decimal - second_input._decimal)

curly_brace = "{"
curly_brace_closing =  "}"
curly_braces = "{{"
curly_braces_closing =  "}}"


total_text = f"""
START_TEST(test_{tested_function_name}) {curly_brace}
// {first_input.decimal_repr}
uint32_t minuend[6] = {curly_brace}{first_input._hex_mantissa_6}{curly_brace_closing};
// {second_input.decimal_repr}
uint32_t subtrahend[6] = {curly_brace}{second_input._hex_mantissa_6}{curly_brace_closing};
// {expected_result.decimal_repr}
uint32_t expected_result[6] = {curly_brace}{expected_result._hex_mantissa_6}{curly_brace_closing};

uint32_t result[6] = {curly_brace}0b0, 0b0, 0b0, 0b0, 0b0, 0b0{curly_brace_closing};

  int is_negative = subtract_long_mantissas(minuend, subtrahend, result);

ck_assert_mem_eq(result, expected_result, sizeof(uint32_t) * 6);
ck_assert_int_eq(is_negative, {expected_result._sign});
{curly_brace_closing}
END_TEST
    """
print(total_text)


### multiplication

In [ ]:
tested_function_name = "mantissas_multiplication"
first_input = MyDecimal.from_string("0")
second_input = MyDecimal.from_string("79228162514264337593543950335")
setcontext(Context(prec=60, rounding=ROUND_HALF_EVEN))

expected_result = MyDecimal(first_input._decimal * second_input._decimal)

curly_brace = "{"
curly_brace_closing =  "}"
curly_braces = "{{"
curly_braces_closing =  "}}"


total_text = f"""
START_TEST(test_{tested_function_name}) {curly_brace}
// {first_input.decimal_repr}
uint32_t term_1[6] = {curly_brace}{first_input._hex_mantissa_6}{curly_brace_closing};
// {second_input.decimal_repr}
uint32_t term_2[6] = {curly_brace}{second_input._hex_mantissa_6}{curly_brace_closing};
// {expected_result.decimal_repr}
uint32_t expected_result[6] = {curly_brace}{expected_result._hex_mantissa_6}{curly_brace_closing};

uint32_t result[6] = {curly_brace}0b0, 0b0, 0b0, 0b0, 0b0, 0b0{curly_brace_closing};
multiply_mantissas(term_1, term_2, result);

ck_assert_mem_eq(result, expected_result, sizeof(uint32_t) * 6);
{curly_brace_closing}
END_TEST
    """
print(total_text)

## Comparison

In [ ]:
list_for_comparison = ['79228162514264337593543950335',
'7922816251426433759354395.5',
'7922816251426433759.950335',
'7922.8162514264337593543950335',
'7.9228162514264337593543950335',
'0.123',
'0.123000000000001',
'0.122999999999999999',
'0.0000000000000000000000000001',
'0.0000000001',
'0',
'333333333.44444444',
'12',
'12.00',
'555555555555555.5555555555',
'-79228162514264337593543950335',
'-7.9228162514264337593543950335',
'3.0',
'-0.123',
'-0.0000000000000000000000000001',
'-0',
'-333333333.44444444',
'-12',
'-555555555555555.55555']


all_permutations = list(permutations(list_for_comparison, 2))

In [ ]:
def get_test_text(all_permutations, test_index, tested_function_name):
    first_decimal = MyDecimal.from_string(all_permutations[test_index][0])
    second_decimal = MyDecimal.from_string(all_permutations[test_index][1])
    curly_brace = "{"
    curly_brace_closing =  "}"
    curly_braces = "{{"
    curly_braces_closing =  "}}"
    comparison_result = str(first_decimal._decimal <= second_decimal._decimal).upper()

    # if first_decimal.__str__()  != first_decimal.decimal_repr:
    #     print(first_decimal.__str__())
    #     print(first_decimal.decimal_repr)
    # if second_decimal.__str__() != second_decimal.decimal_repr:
    #     print(second_decimal.__str__())
    #     print(second_decimal.decimal_repr)
    
    
    test_text = f"""
START_TEST(test_{tested_function_name}_gen_{test_index}) {curly_brace}
// {first_decimal.decimal_repr}
s21_decimal input_decimal_1 = {curly_braces}{first_decimal.full_hex_repr}{curly_braces_closing};
// {second_decimal.decimal_repr}
s21_decimal input_decimal_2 = {curly_braces}{second_decimal.full_hex_repr}{curly_braces_closing};

ComparisonResult s21_result = s21_{tested_function_name}(input_decimal_1, input_decimal_2);
ck_assert_int_eq(s21_result, {comparison_result});
{curly_brace_closing}
END_TEST
    """
    return test_text

In [ ]:
tested_function_name = 'is_less_or_equal'

very_big_test_text = ""


for i in range(len(all_permutations)):
    very_big_test_text+=get_test_text(all_permutations, i, tested_function_name)

with open('tests_body.txt', 'w') as file:
    chars_written_1 = file.write(very_big_test_text)

very_big_addition_text = ""


for i in range(len(all_permutations)):
    addition_text = f"""
tcase_add_test(tc_core, test_{tested_function_name}_gen_{i});"""
    very_big_addition_text+=addition_text


with open('addition.txt', 'w') as file:
    chars_written_2 = file.write(very_big_addition_text)

## Arithmetics

In [ ]:
list_for_div = ['1.2345',
'5.5',
'0.95033',
'79228162514264337593543950335',
'-79228162514264337593543950335',
'7922816251426433.7593543950335',
'7922816251426433.7593543950330',
'1.00000000000049',
'36523494581823720394585.234509',
'-2934834509834059832.45',
'-42353455',
'-11353455324534534597889779',
'4',
'1',
'-1',
'0',
'-0',
'333',                
'12345',
'0.123456789',
'-0.1',
'-300000000',
'1.00000000000101']

all_permutations_for_div = list(permutations(list_for_div, 2))

In [ ]:
from enum import Enum
import operator
class ArithmeticResult(Enum):
  OK = 0
  TOO_BIG = 1
  TOO_SMALL = 2
  DIVISION_BY_ZERO = 3

def perform_operation(first_decimal: Decimal, second_decimal: Decimal, operation: str) -> Decimal:
    op_map = {
        'add': operator.add,
        'sub': operator.sub,
        'mul': operator.mul,
        'div': operator.truediv
    }
    
    if operation not in op_map:
        raise ValueError("Unknown operation")
    
    return op_map[operation](first_decimal, second_decimal)



def get_arithmetics_test_text(all_permutations, test_index, test_func_name):
    myothercontext = Context(prec=29, rounding=ROUND_HALF_EVEN)
    setcontext(myothercontext)
    max_value = Decimal("79228162514264337593543950335")
    min_value = -max_value
    
    first_decimal = MyDecimal.from_string(all_permutations[test_index][0])
    second_decimal = MyDecimal.from_string(all_permutations[test_index][1])
    curly_brace = "{"
    curly_brace_closing =  "}"
    curly_braces = "{{"
    curly_braces_closing =  "}}"

    if second_decimal._decimal == 0 and tested_function_name == 'div':
        expected_return_code = ArithmeticResult.DIVISION_BY_ZERO
    else:
        expected_result = MyDecimal(perform_operation(first_decimal._decimal, 
                                                      second_decimal._decimal, 
                                                      test_func_name))
        if int("".join([str(i) for i in expected_result._decimal.as_tuple().digits])) > max_value:
            myothercontext = Context(prec=28, rounding=ROUND_HALF_EVEN)
            setcontext(myothercontext)
            expected_result = MyDecimal(perform_operation(first_decimal._decimal, 
                                                          second_decimal._decimal, 
                                                          test_func_name))
        expected_result = normalize_exponent(expected_result)

        if expected_result._decimal > max_value:
            expected_return_code = ArithmeticResult.TOO_BIG
        elif expected_result._decimal < min_value:
            expected_return_code = ArithmeticResult.TOO_SMALL
        else:
            expected_return_code = ArithmeticResult.OK
        
    
    if expected_return_code == ArithmeticResult.OK:
        results_part_text = f"""// {expected_result.decimal_repr}
s21_decimal expected_result = {curly_braces}{expected_result.full_hex_repr}{curly_braces_closing};
        
s21_decimal s21_result_decimal; 
ArithmeticResult s21_return_code = s21_{test_func_name}(input_decimal_1, input_decimal_2, &s21_result_decimal);

ck_assert_int_eq(s21_is_equal(s21_result_decimal, expected_result), TRUE);"""
    else:
        results_part_text = f"""
s21_decimal s21_result_decimal; 
ArithmeticResult s21_return_code = s21_{test_func_name}(input_decimal_1, input_decimal_2, &s21_result_decimal);
"""


    total_text = f"""
START_TEST(test_{tested_function_name}_gen_{test_index}) {curly_brace}
// {first_decimal.decimal_repr}
s21_decimal input_decimal_1 = {curly_braces}{first_decimal.full_hex_repr}{curly_braces_closing};
// {second_decimal.decimal_repr}
s21_decimal input_decimal_2 = {curly_braces}{second_decimal.full_hex_repr}{curly_braces_closing};
{results_part_text}
ck_assert_int_eq(s21_return_code, {expected_return_code.name});
{curly_brace_closing}
END_TEST
    """

    return total_text

In [ ]:
tested_function_name = 'mul'
list_to_iterate = all_permutations_for_div
#@@@@@CHANGE HERE@@@@^^^^^^


very_big_test_text = ""
for i in range(len(list_to_iterate)):
    very_big_test_text+=get_arithmetics_test_text(list_to_iterate, i, tested_function_name)

with open('tests_body.txt', 'w') as file:
    written_chars_1 = file.write(very_big_test_text)

very_big_addition_text = ""


for i in range(len(list_to_iterate)):
    addition_text = f"""
tcase_add_test(tc_core, test_{tested_function_name}_gen_{i});"""
    very_big_addition_text+=addition_text


with open('addition.txt', 'w') as file:
    written_chars_2 = file.write(very_big_addition_text)

## conversion

In [ ]:
from enum import Enum

class ConversionResult(Enum):
  OK = 0,
  CONVERSION_ERROR = 1

### int to decimal

In [ ]:
list_for_int_to_decimal = ['-2147483640',
'5',
'-42353455',
'4',
'0',
'-0',
'333',                
'12345',
'-300000000',
'2147483647',
'-2147483648']

In [ ]:
def get_int_to_decimal_test_text(all_permutations, test_index):
    myothercontext = Context(prec=29, rounding=ROUND_HALF_EVEN)
    setcontext(myothercontext)
    
    input_decimal = MyDecimal.from_string(all_permutations[test_index])
    input_decimal = normalize_exponent(input_decimal)
    
    curly_brace = "{"
    curly_brace_closing =  "}"
    curly_braces = "{{"
    curly_braces_closing =  "}}"
    
    expected_return_code = ConversionResult.OK
        
    total_text = f"""
START_TEST(test_from_int_to_decimal_gen_{test_index}) {curly_brace}
int input_int = {all_permutations[test_index]};
// {input_decimal.decimal_repr}
s21_decimal expected_result =  {curly_braces}{input_decimal.full_hex_repr}{curly_braces_closing};

s21_decimal s21_result_decimal;
ConversionResult s21_return_code = s21_from_int_to_decimal(input_int, &s21_result_decimal);

ck_assert_int_eq(s21_is_equal(s21_result_decimal, expected_result), TRUE);
ck_assert_int_eq(s21_return_code, {expected_return_code.name});
{curly_brace_closing}
END_TEST
    """
    return total_text

In [ ]:
very_big_test_text = ""
for i in range(len(list_for_int_to_decimal)):
    very_big_test_text+=get_int_to_decimal_test_text(list_for_int_to_decimal, i)

with open("tests_body.txt", 'w') as file:
    written_chars_1 = file.write(very_big_test_text)

very_big_addition_text = ""


for i in range(len(list_for_int_to_decimal)):
    addition_text = f"""
tcase_add_test(tc_core, test_from_int_to_decimal_gen_{i});"""
    very_big_addition_text+=addition_text


with open('addition.txt', 'w') as file:
    written_chars_2 = file.write(very_big_addition_text)

### decimal to int

In [ ]:
list_for_decimal_to_int = ['-2147483640',
'5',
'-42353455',
'4',
'0',
'-0',
'333',                
'12345',
'-300000000',
'2147483647',
'2147483647.9',
'-2147483649.99999999',
'-2147483648.9999', 
'-1234567.34513',
'7777.777777',
'2147483648.01',
'-2147483648', 
'-2147483649',
'-5555555555555',
'7777777777777777']


In [ ]:
def get_decimal_to_int_test_text(all_permutations, test_index):
    max_int = 2147483647
    min_int = -2147483648
    curly_brace = "{"
    curly_brace_closing =  "}"
    curly_braces = "{{"
    curly_braces_closing =  "}}"
    myothercontext = Context(prec=29, rounding=ROUND_HALF_EVEN)
    setcontext(myothercontext)
    
    input_decimal = MyDecimal.from_string(all_permutations[test_index])

    expected_result = input_decimal._decimal.quantize(Decimal('1.'), rounding=ROUND_DOWN)
    
    if expected_result < min_int or expected_result > max_int:
        expected_return_code = ConversionResult.CONVERSION_ERROR
    else:         
        expected_return_code = ConversionResult.OK

    

    if expected_return_code == ConversionResult.OK:
        results_part_text = f"""int expected_result = {expected_result};
            
int s21_result_int; 
ConversionResult s21_return_code = s21_from_decimal_to_int(input_decimal, &s21_result_int);

ck_assert_int_eq(s21_result_int, expected_result);"""
    else:
        results_part_text = f"""
int s21_result_int; 
ConversionResult s21_return_code = s21_from_decimal_to_int(input_decimal, &s21_result_int);"""
        

    total_text = f"""
START_TEST(test_from_decimal_to_int_gen_{test_index}) {curly_brace}
// {input_decimal.decimal_repr}
s21_decimal input_decimal = {curly_braces}{input_decimal.full_hex_repr}{curly_braces_closing};
{results_part_text}
ck_assert_int_eq(s21_return_code, {expected_return_code.name});
{curly_brace_closing}
END_TEST
    """

    return total_text

In [ ]:
very_big_test_text = ""
for i in range(len(list_for_decimal_to_int)):
    very_big_test_text+=get_decimal_to_int_test_text(list_for_decimal_to_int, i)

with open("tests_body.txt", 'w') as file:
    written_chars_1 = file.write(very_big_test_text)

very_big_addition_text = ""


for i in range(len(list_for_decimal_to_int)):
    addition_text = f"""
tcase_add_test(tc_core, test_from_decimal_to_int_gen_{i});"""
    very_big_addition_text+=addition_text


with open('addition.txt', 'w') as file:
    written_chars_2 = file.write(very_big_addition_text)

### float to decimal

In [ ]:
import re

def degrade_to_cfloat(input_str):
    return MyDecimal.from_string(str(c_float(float(input_str)).value)).decimal_repr 

def strip_dot_zero(input_str):
    return re.sub(r'\.0$', '', input_str)

def get_7_digit_mydecimal(input_float):
    myothercontext = Context(prec=70, rounding=ROUND_HALF_UP)
    input_decimal = Decimal(input_float, context=myothercontext)
    
    sign, digits, exponent = input_decimal.as_tuple()
    
    if len(digits) > 7:
        overflow_digits = len(digits) - 7
    else:
        overflow_digits = 0
    
    mantissa = 0
    for digit in digits[:7]:
        mantissa *= 10
        mantissa += digit
    
    if len(digits) > 7 and digits[7] > 4:
        mantissa+=1
        if mantissa == 10**7:
            overflow_digits+=1
    
    if exponent >= 0:
        normalized = Decimal((sign, [int(i) for i in list(str(mantissa))[:7]], exponent), context=myothercontext)*(10**overflow_digits)
    
    else:
        normalized = Decimal((sign, [int(i) for i in list(str(mantissa))[:7]], exponent+overflow_digits), context=myothercontext)

    return MyDecimal(normalized)


In [ ]:
from ctypes import c_float

list_float_to_decimal = ['1234567800000000',
'1234567812345678',
'1234567123456789',
'1234567200000000',
'9999999123456789',
'9999999912345678',
'0.00000099999991',
'0.000000999999912345',
'0.00000099999995',
'0.000000999999951234',
'0.000000123456712345',
'0.000000123456781234',
'12345.6781234',
'12345.6712345',
'99999.9912345',
'99999.9991234']

# ['1234567813922816.0',
#  '1234567813922816.0',
#  '1234567142834176.0',
#  '1234567142834176.0',
#  '9999999198822400.0',
#  '10000000272564224',
#  '0.000000999999883788405',
#  '0.000000999999883788405',
#  '0.0000009999999974752427',
#  '0.0000009999999974752427',
#  '0.00000012345671507318912', '0.0000001234567861274627', 
#  '12345.677734375', '12345.6708984375',
#  '99999.9921875', '100000.0']
extensive_list = ['1234567800000000','1234567812345678',
'1234567123456789',
'1234567200000000','9999999123456789',
'9999999912345678','0.00000099999991',
'0.000000999999912345','0.00000099999995',
'0.000000999999951234','0.000000123456712345',
'0.000000123456781234','12345.6781234',
'12345.6712345','99999.9912345',
'99999.9991234','1.2345','5.5',
'0.95033','79228162514264337593543950335',
'-79228162514264337593543950335','7922816251426433.7593543950335',
'36523494581823720394585.234509','-2934834509834059832.45',
'-42353455','-11353455324534534597889779','4','0','-0','333',                
'12345','0.123456789','-0.1','-300000000',
'79228162514264337593543950335','7922816251426433759354395.5',
'7922816251426433759.950335','7922.8162514264337593543950335',
'7.9228162514264337593543950335','0.123','0.123000000000001','0.122999999999999999',
'0.0000000000000000000000000001','0.0000000001','0','333333333.44444444',
'12','12.00','555555555555555.5555555555','-79228162514264337593543950335',
'-7.9228162514264337593543950335','3.0','-0.123',
'-0.0000000000000000000000000001','-0',
'-333333333.44444444','-12','-555555555555555.55555']
list_of_degraded_floats = list(set([degrade_to_cfloat(i) for i in set(extensive_list)]))

In [ ]:
MyDecimal.from_string(list_of_degraded_floats[0]).decimal_repr+'f'

In [ ]:
def get_float_to_decimal_test_text(all_permutations, test_index):
    max_value = Decimal("79228162514264337593543950335")
    min_value = Decimal((0,(1,),-28))
    curly_brace = "{"
    curly_brace_closing =  "}"
    curly_braces = "{{"
    curly_braces_closing =  "}}"
    myothercontext = Context(prec=100, rounding=ROUND_HALF_UP)
    setcontext(myothercontext)
    
    input_decimal = MyDecimal.from_string(all_permutations[test_index])
    seven_digit_mydecimal = get_7_digit_mydecimal(all_permutations[test_index])
    expected_result = seven_digit_mydecimal.full_hex_repr
    
    if abs(input_decimal._decimal) < min_value or abs(input_decimal._decimal) > max_value:
        expected_return_code = ConversionResult.CONVERSION_ERROR
    else:         
        expected_return_code = ConversionResult.OK


    if expected_return_code == ConversionResult.OK:
        results_part_text = f"""int expected_result = {expected_result};
            
int s21_result_int; 
ConversionResult s21_return_code = s21_from_decimal_to_int(input_decimal, &s21_result_int);

ck_assert_int_eq(s21_result_int, expected_result);"""
    else:
        results_part_text = f"""
int s21_result_int; 
ConversionResult s21_return_code = s21_from_decimal_to_int(input_decimal, &s21_result_int);"""
        

    total_text = f"""
START_TEST(test_from_decimal_to_int_gen_{test_index}) {curly_brace}
// {input_decimal.decimal_repr}
s21_decimal input_decimal = {curly_braces}{input_decimal.full_hex_repr}{curly_braces_closing};
{results_part_text}
ck_assert_int_eq(s21_return_code, {expected_return_code.name});
{curly_brace_closing}
END_TEST
    """

    return total_text

In [ ]:
myothercontext = Context(prec=100, rounding=ROUND_HALF_UP)
setcontext(myothercontext)

print(get_float_to_decimal_test_text(list_of_degraded_floats, 19))



## other funcs (calculation)

In [ ]:
list_for_other = ['1.2345',
'5.5',
'0.95033',
'79228162514264337593543950335',
'-79228162514264337593543950335',
'7922816251426433.7593543950335',
'36523494581823720394585.234509',
'-2934834509834059832.45',
'-42353455',
'-11353455324534534597889779',
'4',
'0',
'-0',
'333',                
'12345',
'0.123456789',
'-0.1',
'-300000000']

In [ ]:
list_for_truncate = ['1.2345678910111213141516123235',
'5.50',
'-5.500',
'0.95033',
'792281625142643375935439503.9',
'7922816251426433759354395033.4',
'-792281625142643375935439503.9',
'7922816251426433759354395033.1',
'7.9228162514264337593543950335',
'-7922816251426433759354395033.5',
'7922816251426433.7593543950335',
'36523494581823720394585.234509',
'-2934834509834059832.45',
'-42.353455',
'-11353455324534534597889.779',
'4',
'0',
'-0',
'333',                
'12345',
'0.123456789',
'-0.1',
'-300000000']

In [ ]:
def negate(input_mydecimal):
    myothercontext = Context(prec=29, rounding=ROUND_HALF_EVEN)
    setcontext(myothercontext)
    expected_result = input_mydecimal._decimal * (-1)
    return MyDecimal(expected_result)
    
def truncate(input_mydecimal):
    expected_result = input_mydecimal._decimal.quantize(Decimal('1.'), rounding=ROUND_DOWN)
    return MyDecimal(expected_result)

def floor(input_mydecimal):
    expected_result = input_mydecimal._decimal.to_integral_value(rounding=ROUND_FLOOR)
    return MyDecimal(expected_result)

def round_my_decimal(input_mydecimal):
    expected_result = input_mydecimal._decimal.to_integral_value(rounding=ROUND_HALF_UP)
    return MyDecimal(expected_result)


def perform_operation(input_mydecimal: MyDecimal, operation: str) -> MyDecimal:
    op_map = {
        'negate': negate,
        'truncate': truncate,
        'floor': floor,
        'round': round_my_decimal
    }
    
    if operation not in op_map:
        raise ValueError("Unknown operation")
    
    return op_map[operation](input_mydecimal)


In [ ]:
from enum import Enum

class CalculationResult(Enum):
  OK = 0
  CALCULATION_ERROR = 1

def get_other_test_text(all_permutations, test_index, test_func_name):
    myothercontext = Context(prec=29, rounding=ROUND_HALF_EVEN)
    setcontext(myothercontext)
    
    input_decimal = MyDecimal.from_string(all_permutations[test_index])

    curly_brace = "{"
    curly_brace_closing =  "}"
    curly_braces = "{{"
    curly_braces_closing =  "}}"
    
    # expected_result = MyDecimal(input_decimal._decimal * (-1))
    expected_result = perform_operation(input_decimal, test_func_name)
    expected_result = normalize_exponent(expected_result)
    expected_return_code = CalculationResult.OK
    results_part_text = f"""// {expected_result.decimal_repr}
s21_decimal expected_result = {curly_braces}{expected_result.full_hex_repr}{curly_braces_closing};
        
s21_decimal s21_result_decimal; 
CalculationResult s21_return_code = s21_{test_func_name}(input_decimal_1, &s21_result_decimal);

ck_assert_int_eq(s21_is_equal(s21_result_decimal, expected_result), TRUE);"""


    total_text = f"""
START_TEST(test_{tested_function_name}_gen_{test_index}) {curly_brace}
// {input_decimal.decimal_repr}
s21_decimal input_decimal_1 = {curly_braces}{input_decimal.full_hex_repr}{curly_braces_closing};
{results_part_text}
ck_assert_int_eq(s21_return_code, {expected_return_code.name});
{curly_brace_closing}
END_TEST
    """

    return total_text

In [ ]:
tested_function_name = 'round'
list_to_iterate = list_for_truncate
#@@@@@CHANGE HERE@@@@^^^^^^

very_big_test_text = ""
for i in range(len(list_to_iterate)):
    very_big_test_text+=get_other_test_text(list_to_iterate, i, tested_function_name)

with open("tests_body.txt", 'w') as file:
    written_chars_1 = file.write(very_big_test_text)

very_big_addition_text = ""


for i in range(len(list_to_iterate)):
    addition_text = f"""
tcase_add_test(tc_core, test_{tested_function_name}_gen_{i});"""
    very_big_addition_text+=addition_text


with open('addition.txt', 'w') as file:
    written_chars_2 = file.write(very_big_addition_text)

In [ ]:
a = MyDecimal.from_string('4.12345678910111213141516')

myothercontext = Context(prec=29, rounding=ROUND_HALF_UP)


# myothercontext = Context(prec=29, rounding=ROUND_HALF_EVEN)

# with localcontext(ctx=myothercontext):
#     res = a._decimal - b._decimal
#     MyDecimal(res)

In [ ]:
a = MyDecimal.from_string('1')

In [ ]:
int(a._decimal) #.full_hex_repr

In [ ]:
myothercontext = Context(prec=29, rounding=ROUND_HALF_EVEN)
with localcontext(ctx=myothercontext):
    res = (MyDecimal.from_string('79228162514264337593543950335')._decimal / MyDecimal.from_string('-0.0000000000000000000000000001')._decimal)

    # print(res)
    # z = res.as_tuple()
    sign, digits, exponent = res.as_tuple()
    res
    exponent
    # z = digits 
    # "".join(str(digits))

    

In [ ]:
setcontext(Context(prec=28, rounding=ROUND_HALF_EVEN))
MyDecimal.from_string("7922816251426433.7593543950335")._decimal + MyDecimal.from_string("245.32")._decimal

setcontext(Context(prec=28, rounding=ROUND_HALF_EVEN))
MyDecimal.from_string("20668995.773191220928874517498")._decimal + MyDecimal.from_string("58560166.7410")._decimal

setcontext(Context(prec=28, rounding=ROUND_HALF_EVEN))
a = (MyDecimal.from_string("7922816251426433.7593543950335")._decimal + MyDecimal.from_string("245.32")._decimal)
int("".join([str(i) for i in a.as_tuple().digits])) > 79228162514264337593543950335

In [ ]:
def make_service_part_string(power):

    service_part_string = 8*'0'+bin(power)[2:].zfill(8)+(16*'0')
    return service_part_string

a = [print(f"{i:2}: 0b{make_service_part_string(i)}") for i in range(29)]

In [ ]:
def count_decimal_places(number):
    str_number = str(number).rstrip('0')
    
    if '.' in str_number:
        if str_number.endswith('.'):
            return 0

        integer_part, decimal_part = str_number.split('.')
        

        return len(decimal_part)
    else:
        return 0

test_numbers = [123.54, 123.3, 123.345, 123, 123.0, 123.10, 123.100, 123.00000, 123.10000, 123.10001]

for num in test_numbers:
    print(f"{num}: {count_decimal_places(num)} decimal places")